# List of contents:

  1. Download files
  2. Configuration
  3. Queries building
  4. Run experiment
  5. Evaluation

The experiment takes some hours to finish in Google Colab, selecting the GPU as accelerator. In case the user would like to obtain the evaluation of the results without executing the whole experiment, the blocks 2 and 4 can be skiped (the file with the results obtained by GPT-2 in our execution is downloaded in block 1). 

# 1. Download files

In [ ]:
#Download the necessary files for the experiment

!pip install googledrivedownloader

from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1SKZNH3vC3O1U7xVCOEbWskjR0ajG3-if',
                                    dest_path='./content',
                                    unzip=True)

Unzipping...Done.


# 2. Configuration

In [ ]:
#Install Tensorflow

!pip3 install tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 22kB/s 
     |████████████████████████████████| 51kB 5.1MB/s 
     |████████████████████████████████| 512kB 38.4MB/s 
     |████████████████████████████████| 3.8MB 21.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7557 sha256=32ac4257ce95dfaed2e85c6968468ef50c9ab99bbfc23abfb3bf60383a407043
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.15.0 which is incompatible.
  Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0


In [ ]:
#Install requirements

%cd gpt-2
!pip3 install -r requirements.txt

[Errno 2] No such file or directory: 'gpt-2'
/content
     |████████████████████████████████| 92kB 2.8MB/s 
     |████████████████████████████████| 604kB 5.2MB/s 
     |████████████████████████████████| 61kB 4.9MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 61kB 6.5MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=6bafe333c0a8a68fc4586728826cc5e9ea9f2da498456c2c67d70ee9ba355563
  Stored in directory: /root/.cache/pip/wheels/af/19/30/1ea0cad502dcb4e66ed5a690279628c827aea38bbbab75d5ed
  Created wheel for regex: filename=regex-2017.4.5-cp37-cp37m-linux_x86_64.whl size=534413 sha256=c30d988365c7440a2190c5e4c94a44d39304240012c89777ac5500aefc470828
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.31.1 which is incompatible.
ERROR: kap

In [ ]:
#More GPT2 requirements before running the model

%cd gpt-2
!export PYTHONIOENCODING=UTF-8

[Errno 2] No such file or directory: 'gpt-2'
/content


In [ ]:
#Download the desired model (in this experiment we will use the 774M parameters model)

!python3 download_model.py 774M

Fetching checkpoint: 1.00kit [00:00, 621kit/s]                                                      
Fetching encoder.json: 1.00kit [00:00, 554kit/s]                                                    
Fetching hparams.json: 1.00kit [00:00, 750kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.00kit [00:00, 775kit/s]                                  
Fetching model.ckpt.index: 1.00kit [00:00, 733kit/s]                                                
Fetching model.ckpt.meta: 1.00kit [00:00, 707kit/s]                                                 
Fetching vocab.bpe: 1.00kit [00:00, 767kit/s]                                                       


# 2. Queries building

In [9]:
#These lines will create the GPT2 inputs (or queries) in the desired form

numConcepts = 8
numBatches = 3
numTests = 5

queries = [[]]*numConcepts
witnessSets = [[('0', '0')],[('10', '0')],[('', '0'), ('01', '')],[('0', '011')],[('', '1'), ('1110', '')],[('0', '10'), ('00', '')],[('', '0'), ('0', ''), ('00', '0')],[('', '01'), ('01', ''), ('1', '')]]
additionalSets1 = [[('111001', '1'), ('110101', '1')],[('01000', '1'), ('01010', '1')],[('010100', ''), ('10101', '1')],[('10', '1'), ('001', '011')],[('10', '1'), ('111001', '')],[('11', '01'), ('10', '')],[('0001', '0'), ('01', '1')],[('11', ''), ('011', '')]]
additionalSets2 = [[('100110', '1'), ('111100', '1'), ('01010', '0')],[('001', '0'), ('00', '0'), ('1011', '0')],[('010', ''), ('100', '1'), ('011101', '')],[('0001', '011'), ('00', '011'), ('000', '011')],[('11', '1'), ('0001', '0'), ('11100', '')],[('', '1'), ('101', ''), ('000', '')],[('0101', '1'), ('0010', '0'), ('0110', '1')],[('10', ''), ('0100', ''), ('0', '0')]]
testSets = [[('00000', '0'), ('11100', '1'), ('00111', '0'), ('11010', '1'), ('0010', '0')], [('01011', '1'), ('0101', '1'), ('0010', '0'), ('100', '0'), ('1', '-')], [('00010', '1') , ('110', '-'), ('00111', '1'), ('11000', '-'), ('101', '1')], [('01011', '011'), ('0101', '011'), ('0010', '011'), ('100', '1'), ('1', '1')], [('01011', '0'), ('110', '1'), ('0010', '-'), ('101', '1'), ('1000', '-')], [('01', '00'), ('0000', '-'), ('00011', '-'), ('0011', '-'), ('1000', '-')], [('01011', '1'), ('0000', '0'), ('00000', '0'), ('100', '-'), ('1000', '-')], [('10101', '-'), ('11101', '-'), ('00000', '1'), ('0011', '1'), ('1111', '-')]]

#There will be a total of 8*3*5=120  different queries for GPT2 (8 concepts * 3 learning phases * 5 test examples to complete)

#A list with 120 positions (queries) is created. The followed procedure is to store the query for the witness set 
#of the first concept and the first test example in the first position, for the second test example in the second position...

#The query for the WS + first redundant examples of the first concept and the first test example will be stored
#in the 6th position...

#The query for the WS of the second concept and the first test example will be stored 
#in the 16th position...

def createQueries():
  for c in range(0, numConcepts):
    queries[c] = [[]]*numBatches
    #Variable to count the number of examples in a query
    num = 1
    #Variable to store the query examples before adding the test examples
    base = ''
    for b in range(0, numBatches):
      queries[c][b] = [""]*numTests
      for t in range(0, numTests):
        if b == 0:
          for w in range(0, len(witnessSets[c])):
            if witnessSets[c][w][0] == "":
              input1 = "-"
            else:
              input1 = witnessSets[c][w][0]

            if witnessSets[c][w][1] == "":
              output1 = "-"
            else:
              output1 = witnessSets[c][w][1]
            base = base + "Input" + str(num) + ": " + input1 + ", Output" + str(num) + ": " + output1 + "; "
            num = num + 1
          queries[c][b][t] = base + "Input" + str(num) + ": " + testSets[c][t][0] + ", Output" + str(num) + ":"
        
        elif b == 1:
          for a1 in range(0, len(additionalSets1[c])):
            if additionalSets1[c][a1][0] == "":
              input1 = "-"
            else:
              input1 = additionalSets1[c][a1][0]

            if additionalSets1[c][a1][1] == "":
              output1 = "-"
            else:
              output1 = additionalSets1[c][a1][1]
            base = base + "Input" + str(num) + ": " + input1 + ", Output" + str(num) + ": " + output1 + "; "
            num = num + 1
          queries[c][b][t] = base + "Input" + str(num) + ": " + testSets[c][t][0] + ", Output" + str(num) + ":"
        
        elif b == 2:
          for a2 in range(0, len(additionalSets2[c])):
            if additionalSets2[c][a2][0] == "":
              input1 = "-"
            else:
              input1 = additionalSets2[c][a2][0]

            if additionalSets2[c][a2][1] == "":
              output1 = "-"
            else:
              output1 = additionalSets2[c][a2][1]
            base = base + "Input" + str(num) + ": " + input1 + ", Output" + str(num) + ": " + output1 + "; "
            num = num + 1
          queries[c][b][t] = base + "Input" + str(num) + ": " + testSets[c][t][0] + ", Output" + str(num) + ":"


createQueries()
print(queries)

[[['Input1: 0, Output1: 0; Input2: 00000, Output2:', 'Input1: 0, Output1: 0; Input2: 0, Output2: 0; Input3: 11100, Output3:', 'Input1: 0, Output1: 0; Input2: 0, Output2: 0; Input3: 0, Output3: 0; Input4: 00111, Output4:', 'Input1: 0, Output1: 0; Input2: 0, Output2: 0; Input3: 0, Output3: 0; Input4: 0, Output4: 0; Input5: 11010, Output5:', 'Input1: 0, Output1: 0; Input2: 0, Output2: 0; Input3: 0, Output3: 0; Input4: 0, Output4: 0; Input5: 0, Output5: 0; Input6: 0010, Output6:'], ['Input1: 0, Output1: 0; Input2: 0, Output2: 0; Input3: 0, Output3: 0; Input4: 0, Output4: 0; Input5: 0, Output5: 0; Input6: 111001, Output6: 1; Input7: 110101, Output7: 1; Input8: 00000, Output8:', 'Input1: 0, Output1: 0; Input2: 0, Output2: 0; Input3: 0, Output3: 0; Input4: 0, Output4: 0; Input5: 0, Output5: 0; Input6: 111001, Output6: 1; Input7: 110101, Output7: 1; Input8: 111001, Output8: 1; Input9: 110101, Output9: 1; Input10: 11100, Output10:', 'Input1: 0, Output1: 0; Input2: 0, Output2: 0; Input3: 0, Outp

# 3. Run experiment

In [ ]:
#The total outputs required are equal to 8 * 3 * 5 * 40 (concepts number * learning phases number * test examples number * number of same-query executions 
#or repetitions) = 4800. The execution time took more than 4 hours, so it was preferable to persist the results in a file rather than a list in the 
#python environment.  

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Run GPT-2
 
numReps = 40
 
from src import nointeractive as ni
 
filePath = '/content/drive/My Drive/GPT2results.txt'
 
ni.rungpt2(queries, numReps, filePath, numConcepts, numBatches, numTests)

# 4. Evaluation

Index of results variables:

- "accuracies_ens" and "binary_ens" -> gpt-2-ens
- "accuracies_ens_emp" and "binary_ens_emp" -> gpt-2-ens without test examples with empty string as output
- "accuracies_exp" and "binary_exp" -> gpt-2-ens
- "accuracies_exp_emp" and "binary_exp_emp" -> gpt-2-exp without test examples with empty string as output



In [5]:
#Common variables

numConcepts = 8
numBatches = 3
numTests = 5
numReps = 40

seed = 23

## 4.1 Data cleaning and variables preparation

In [ ]:
#Upload results file generated with the previous code snippet

#The file with the results is already included in the zip downloaded from the Drive folder, so we can avoid
#the long execution everytime we want to postprocess the results

#Uncomment the following lines in case of running the full experiment and wanting to upload the results file generated

#from google.colab import files
#uploaded = files.upload()

In [1]:
#Create a variable with the results

%cd gpt-2
resdata = open("GPT2results.txt",encoding="UTF-8").read().split("\n")

[WinError 2] The system cannot find the file specified: 'gpt-2'
C:\Users\jaimovitchg\Desktop\code\GPT-2 Experiments


In [2]:
#Data cleaning
resdata = [x for x in resdata if x[:7] == 'Concept']
for i in range(0, len(resdata)):
  resdata[i] = resdata[i].split('//')

for i in range(0, len(resdata)):
  resdata[i][2] = resdata[i][2].split('Output= ')
  resdata[i][2].pop(0)
  resdata[i][2][0] = resdata[i][2][0].lstrip(' ')
  resdata[i][2] = resdata[i][2][0]

In [3]:
#Outputs cleaning
for i in range(0, len(resdata)):
  for j in range(0, len(resdata[i][2])):
    if resdata[i][2][j] != '1' and resdata[i][2][j] != '0':
      if j == 0:
        if resdata[i][2][j] == '-':  
          resdata[i][2] = "-"
        else:
          resdata[i][2] = ''
      else:
        resdata[i][2] = resdata[i][2][:j]
      break

In [6]:
#Store information in a more comprehensive variable finaldata

finaldata = [[]]*numReps

for r in range(0, numReps):
  finaldata[r] = [[]]*numConcepts
  for c in range(0,numConcepts):
    finaldata[r][c] = [[]]*numBatches
    for b in range(0,numBatches):
      finaldata[r][c][b] = [[]]*numTests
      for t in range(0, numTests):
        finaldata[r][c][b][t] = resdata[c*numBatches*numTests*numReps + b*numTests*numReps + t*numReps + r][2]

In [12]:
#Store information in more comprehensive variables finaldata_emp (no test outputs with empty string) and testSets_emp

import copy

finaldata_emp = copy.deepcopy(finaldata)
testSets_emp = copy.deepcopy(testSets)

remove_pos = [[],[4],[1,3],[],[2,4],[1,2,3,4],[3,4],[0,1,4]]


for i in range(0, len(remove_pos)):
    remove_pos[i].reverse()
    for j in range(0,len(remove_pos[i])):
        del testSets_emp[i][remove_pos[i][j]]
        for b in range(0, numBatches):
            for r in range(0, numReps):
                del finaldata_emp[r][i][b][remove_pos[i][j]]

## 4.2 GPT-2-ens evaluation

In [13]:
#MOST FREQUENT OUTPUT (IN CASE OF TIES, RANDOM SELECTION WITH SEED 23) 

import random as rn

#https://www.geeksforgeeks.org/python-find-most-frequent-element-in-a-list/ addapted

#Return the most repeated element in a list. If there are equal frequence of the top answers, return random result
def most_frequent(passed_list): 
  counter = 0 
      
  for i in passed_list: 
    curr_frequency = passed_list.count(i) 
    if curr_frequency > counter: 
      counter = curr_frequency
      num = i 
    elif curr_frequency == counter:
      if isinstance(num, list):
        if not i in num:
          num.append(i)
      elif i != num:
        num = [num,i]
  
  return num 


#Collect the different outputs provided by GPT2 for a test example and call most_frequent to obtain the most frequent output or the list with the most
#frequent ones in case of ties. In the later case, perform random selection.            
aggregAnswers = [[]]*numConcepts
for c in range(0, numConcepts):
  aggregAnswers[c] = [[]]*numBatches
  for b in range(0,numBatches):
    aggregAnswers[c][b] = ['']*numTests
    for t in range(0,numTests):
      checklist = list('')
      for r in range(0,numReps):
        if finaldata[r][c][b][t] != '':
          checklist.append(finaldata[r][c][b][t])
      aggregAnswers[c][b][t]= most_frequent(checklist)
      if isinstance(aggregAnswers[c][b][t], list):
         rn.seed(seed)
         aggregAnswers[c][b][t] = rn.choice(aggregAnswers[c][b][t])


#Will be used to perform the similarity analysis
binary_ens = [[]]*numConcepts
#Calculate the accuracies
accuracies_ens = [[]]*numConcepts
for c in range(0, numConcepts):
  binary_ens[c] = [[]]*numBatches
  accuracies_ens[c] = [0]*numBatches
  for b in range(0, numBatches):
    binary_ens[c][b] = ['']*numTests
    for t in range(0, numTests):
        if aggregAnswers[c][b][t] == testSets[c][t]:
          accuracies_ens[c][b] = accuracies_ens[c][b] + (1/numTests)
          binary_ens[c][b][p] = 1 
        else:
          binary_ens[c][b][p] = 0



## 4.3 GPT-2-ens (NO TEST EXAMPLES WITH EMPTY STRING AS OUTPUT) evaluation

In [17]:
#Additional code for the case when examples with the empty string as output are not allowed

#Return the most repeated element in a list. If there are equal frequence of the top answers, return random result
def most_frequent(passed_list): 
  counter = 0 
      
  for i in passed_list: 
    curr_frequency = passed_list.count(i) 
    if curr_frequency > counter: 
      counter = curr_frequency
      num = i 
    elif curr_frequency == counter:
      if isinstance(num, list):
        if not i in num:
          num.append(i)
      elif i != num:
        num = [num,i]
  
  return num 

#Collect the different outputs provided by GPT2 for a test example and call most_frequent to obtain the most frequent output or the list with the most
#frequent ones in case of ties. In the later case, perform random selection.            
aggregAnswers = [[]]*numConcepts
for c in range(0, numConcepts):
  aggregAnswers[c] = [[]]*numBatches
  for b in range(0,numBatches):
    aggregAnswers[c][b] = ['']*len(testSets_emp[c])
    for t in range(0,len(testSets_emp[c])):
      checklist = list('')
      for r in range(0,numReps):
        if finaldata_emp[r][c][b][t] != '':
          checklist.append(finaldata_emp[r][c][b][t])
      aggregAnswers[c][b][t]= most_frequent(checklist)
      if isinstance(aggregAnswers[c][b][t], list):
         rn.seed(seed)
         aggregAnswers[c][b][t] = rn.choice(aggregAnswers[c][b][t])


#Will be used to perform the similarity analysis
binary_ens_emp = [[]]*numConcepts
#Calculate the accuracies
accuracies_ens_emp = [[]]*numConcepts
for c in range(0, numConcepts):
  binary_ens_emp[c] = [[]]*numBatches
  accuracies_ens_emp[c] = [0]*numBatches
  for b in range(0, numBatches):
    binary_ens_emp[c][b] = ['']*len(testSets_emp[c])
    for t in range(0, len(testSets_emp[c])):
        if aggregAnswers[c][b][p] == testSets_emp[c][t]:
          accuracies_ens_emp[c][b] = accuracies_ens_emp[c][b] + (1/len(testSets_emp[c]))
          binary_ens_emp[c][b][t] = 1
        else:
          binary_ens_emp[c][b][t] = 0


## 4.4 GPT-2-exp evaluation

In [18]:
#AVERAGE ACCURACY WITH N = 40 GPT2 MACHINES

#Will be used to perform the similarity analysis
binary_exp = [[]]*numReps
#Calculate the accuracies
accuracies_exp = [[]]*numReps
for r in range(0, numReps):
  binary_exp[r] = [[]]*numConcepts
  accuracies_exp[r] = [[]]*numConcepts
  for c in range(0, numConcepts):
    accuracies_exp[r][c] = [0]*numBatches
    binary_exp[r][c] = [[]]*numBatches
    for b in range(0, numBatches):
      binary_exp[r][c][b] = ['']*numTests
      for t in range(0, numTests):
          if finaldata[r][c][b][t] == testSets[c][t]:
            accuracies_exp[r][c][b] = accuracies_exp[r][c][b] + (1/numTests)
            binary_exp[r][c][b][t] = 1
          else:
            binary_exp[r][c][b][t] = 0


## 4.5 GPT-2 exp (NO TEST EXAMPLES WITH EMPTY STRING AS OUTPUT) evaluation

In [20]:
#Will be used to perform the similarity analysis
binary_exp_emp = [[]]*numReps
#Calculate the accuracies
accuracies_exp_emp = [[]]*numReps
for r in range(0, numReps):
  binary_exp_emp[r] = [[]]*numConcepts
  accuracies_exp_emp[r] = [[]]*numConcepts
  for c in range(0, numConcepts):
    accuracies_exp_emp[r][c] = [0]*numBatches
    binary_exp_emp[r][c] = [[]]*numBatches
    for b in range(0, numBatches):
      binary_exp_emp[r][c][b] = ['']*len(testSets_emp[c])
      for t in range(0, len(testSets_emp[c])):
          if finaldata_emp[r][c][b][t] == testSets_emp[c][t]:
            accuracies_exp_emp[r][c][b] = accuracies_exp_emp[r][c][b] + (1/len(testSets_emp[c]))
            binary_exp_emp[r][c][b][t] = 1
          else:
            binary_exp_emp[r][c][b][t] = 0